# Data visualization for reforestation dataset

### Load libraries

In [ ]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import shape
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely.geometry import shape
import numpy as np

### Read data

In [ ]:



gdf_dict = json.loads(gdf_filtered.to_json())


for feature, row in zip(gdf_dict['features'], gdf_filtered.itertuples()):
    if 'properties' in feature:
        feature['properties']['Top_Three_NDVI_Months'] = row.Top_Three_NDVI_Months


geojson_str = json.dumps(gdf_dict)


with open("../input/df_reforestation.geojson", "w") as file:
    file.write(geojson_str)

In [ ]:
projects = gpd.read_file('df_reforestation.geojson')
projects.info()

In [ ]:


df_reforestation=gdf_filtered

country_counts = df_reforestation.groupby('country').size().reset_index(name='counts')

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


world_with_counts = world.merge(country_counts, how="left", left_on="name", right_on="country")


world_with_counts['counts'] = world_with_counts['counts'].fillna(0)


fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world_with_counts.plot(column='counts', ax=ax, legend=True,
                       legend_kwds={'label': "Count of Reforestation Sites by Country",
                                    'orientation': "horizontal"},
                       missing_kwds={"color": "lightgrey", "label": "No data"})
plt.show()

In [ ]:


def convert_to_geometry(geojson_input):
    if geojson_input == '{}' or geojson_input is None:
        return None
    try:
        if isinstance(geojson_input, dict): 
            return shape(geojson_input)
        else:
            geojson_obj = json.loads(geojson_input) 
            return shape(geojson_obj)
    except (json.JSONDecodeError, TypeError):
        return None

df_reforestation['geometry'] = df_reforestation['geometry_reported'].apply(convert_to_geometry)
gdf_sites = gpd.GeoDataFrame(df_reforestation, geometry='geometry')




world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


gdf_sites_with_country = gpd.sjoin(gdf_sites, world, how="left", op='intersects')


country_counts_from_sites = gdf_sites_with_country.groupby('name').size().reset_index(name='counts')


country_counts = country_counts.merge(country_counts_from_sites, how="outer", left_on="country", right_on="name")
country_counts['counts'] = country_counts[['counts_x', 'counts_y']].sum(axis=1)
country_counts.drop(['counts_x', 'counts_y', 'name'], axis=1, inplace=True)

world_with_counts = world.merge(country_counts, how="left", left_on="name", right_on="country")


world_with_counts['counts'] = world_with_counts['counts'].fillna(0)


fig = px.choropleth(world_with_counts,
                    locations="iso_a3",
                    color="counts",
                    hover_name="name",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title="Global Reforestation Sites Count")

fig.show()

In [ ]:



fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.7, 0.3],
    specs=[[{"type": "choropleth"}, {"type": "table"}]]
)


fig.add_trace(
    go.Choropleth(
        locations=world_with_counts['iso_a3'],
        z=world_with_counts['counts'],
        hoverinfo="location+name+z",
        colorscale=px.colors.sequential.Plasma,
        colorbar_title="Site Counts"
    ),
    row=1, col=1
)


table_data = world_with_counts[['name', 'counts']].sort_values(by='counts', ascending=False)


fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Counts"],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[table_data.name, table_data.counts],
                   fill_color='lavender',
                   align='left')
    ),
    row=1, col=2
)


fig.update_layout(
    title_text="Global Reforestation Sites Count with Table",
    width=1200,
    height=600
)


fig.show()

In [ ]:

gdf_sites_with_country['site_count'] = 1  
country_site_count = gdf_sites_with_country.groupby('name')['site_count'].sum().reset_index(name='total_sites')


world_with_site_count = world.merge(country_site_count, how="left", left_on="name", right_on="name")
world_with_site_count['total_sites'] = world_with_site_count['total_sites'].fillna(0).astype(int)



fig = make_subplots(rows=1, cols=2, column_widths=[0.8, 0.2], specs=[[{"type": "choropleth"}, {"type": "table"}]])

world_with_site_count['hover_text'] = world_with_site_count['name'] + '<br>Total Sites: ' + world_with_site_count['total_sites'].astype(str)


fig.add_trace(
    go.Choropleth(
        locations=world_with_site_count['iso_a3'],
        z=world_with_site_count['total_sites'],
        hoverinfo='text',  
        text=world_with_site_count['hover_text'], 
        colorscale=px.colors.sequential.Plasma,
        colorbar_title="Total Sites",
    ),
    row=1, col=1
)


table_data = world_with_site_count[['name', 'total_sites']].sort_values(by='total_sites', ascending=False)

header_color = 'rgb(12, 7, 134)'  
cell_color = 'rgb(240, 249, 33)' 


fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Total Sites"],
                    fill_color=header_color,  
                    font=dict(color='white'),
                    align='left'),
        cells=dict(values=[table_data.name, table_data.total_sites],
                   fill_color=cell_color, 
                   align='left')),
    row=1, col=2
)


fig.update_layout(
    title_text="Global Reforestation Sites Count",
    showlegend=False,
    paper_bgcolor="lightgrey",
    title_x=0.5,
    width=1200,
    height=600,
    margin=dict(l=10, r=10, t=50, b=10)
)


fig.show()

In [ ]:

dark_green_scale = [
    [0, '#a1d99b'],  
    [0.5, '#238b45'],  
    [1, '#00441b']  
]

fig.add_trace(
    go.Choropleth(
        locations=world_with_site_count['iso_a3'],
        z=world_with_site_count['total_sites'],
        hoverinfo='text',  
        text=world_with_site_count['hover_text'], 
        colorscale=dark_green_scale,  
        colorbar_title="Total Sites",
    ),
    row=1, col=1
)


header_color = '#238b45'  
cell_color = '#a1d99b'  

fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Total Sites"],
                    fill_color=header_color, 
                    font=dict(color='white'),
                    align='left'),
        cells=dict(values=[table_data.name, table_data.total_sites],
                   fill_color=cell_color,  
                   align='left')),
    row=1, col=2
)


fig.update_layout(
    title_text="Global Reforestation Sites Count",
    showlegend=False,
    paper_bgcolor="lightgrey",
    title_x=0.5,
    width=1200,
    height=600,
    margin=dict(l=10, r=10, t=50, b=10)
)

fig.show()

In [ ]:


def convert_to_geometry(geojson_input):
    if geojson_input == '{}' or geojson_input is None:
        return None
    try:
        if isinstance(geojson_input, dict):  
            return shape(geojson_input)
        else:
            geojson_obj = json.loads(geojson_input)  
            return shape(geojson_obj)
    except (json.JSONDecodeError, TypeError):
        return None


df_reforestation['geometry'] = df_reforestation['geometry_reported'].apply(convert_to_geometry)
gdf_sites = gpd.GeoDataFrame(df_reforestation, geometry='geometry')


gdf_sites.crs = "EPSG:4326"

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


gdf_sites_with_country = gpd.sjoin(gdf_sites, world, how="left", predicate='intersects')


gdf_sites_with_country['area_km2'] = gdf_sites_with_country.to_crs(epsg=3395).area / 10**6


country_area_covered = gdf_sites_with_country.groupby('name')['area_km2'].sum().reset_index(name='total_area_km2')

world_with_area_covered = world.merge(country_area_covered, how="left", left_on="name", right_on="name")


world_with_area_covered['total_area_km2'] = world_with_area_covered['total_area_km2'].fillna(0)


dark_green_scale = [
    [0, '#a1d99b'],  
    [0.5, '#238b45'],  
    [1, '#00441b']  
]

fig = px.choropleth(world_with_area_covered,
                    locations="iso_a3",
                    color="total_area_km2",
                    hover_name="name",
                    color_continuous_scale=dark_green_scale,  # Use the custom dark green scale
                    title="Global Reforestation Area Covered (km²)")

fig.show()

In [ ]:

world_with_area_covered['total_area_km2'] = world_with_area_covered['total_area_km2'].round(3)


fig = make_subplots(rows=1, cols=2, column_widths=[0.8, 0.2], specs=[[{"type": "choropleth"}, {"type": "table"}]])


world_with_area_covered['hover_text'] = world_with_area_covered['name'] + '<br>Total Area: ' + world_with_area_covered['total_area_km2'].astype(str) + ' km²'

fig.add_trace(
    go.Choropleth(
        locations=world_with_area_covered['iso_a3'],
        z=world_with_area_covered['total_area_km2'],
        hoverinfo='text',  
        text=world_with_area_covered['hover_text'], 
        colorscale=dark_green_scale,
        colorbar_title="Total Area<br>km²",
    ),
    row=1, col=1
)

table_data = world_with_area_covered[['name', 'total_area_km2']].sort_values(by='total_area_km2', ascending=False)
fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Total Area Covered (km²)"],
                    fill_color='darkgreen',  
                    font=dict(color='white'),
                    align='left'),
        cells=dict(values=[table_data.name, table_data.total_area_km2],
                   fill_color='palegreen',  
                   align='left')),
    row=1, col=2
)


fig.update_layout(
    title_text="Global Reforestation Area Covered (km²) with Data Table",
    showlegend=False,
    paper_bgcolor="lightgrey",
    title_x=0.5,
    width=1200,
    height=600,
    margin=dict(l=10, r=10, t=50, b=10)
)

fig.show()

In [ ]:

world_with_area_covered['total_area_km2'] = world_with_area_covered['total_area_km2'].round(3)


fig = make_subplots(rows=1, cols=2, column_widths=[0.8, 0.2], specs=[[{"type": "choropleth"}, {"type": "table"}]])


world_with_area_covered['hover_text'] = world_with_area_covered['name'] + '<br>Total Area: ' + world_with_area_covered['total_area_km2'].astype(str) + ' km²'

fig.add_trace(
    go.Choropleth(
        locations=world_with_area_covered['iso_a3'],
        z=world_with_area_covered['total_area_km2'],
        hoverinfo='text',
        text=world_with_area_covered['hover_text'], 
        colorscale=px.colors.sequential.Viridis,
        colorbar_title="Total Area<br>km²",
    ),
    row=1, col=1
)


table_data = world_with_area_covered[['name', 'total_area_km2']].sort_values(by='total_area_km2', ascending=False)
fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Total Area Covered (km²)"],
                    fill_color='#440154', 
                    font=dict(color='white'),
                    align='left'),
        cells=dict(values=[table_data.name, table_data.total_area_km2],
                   fill_color='#b8de29',  
                   align='left')),
    row=1, col=2
)

fig.update_layout(
    title_text="Global Reforestation Area Covered (km²) with Data Table",
    showlegend=False,
    paper_bgcolor="lightgrey",
    title_x=0.5,
    width=1200,
    height=600,
    margin=dict(l=10, r=10, t=50, b=10)
)

fig.show()

In [ ]:

world_with_area_covered['total_area_km2'] = world_with_area_covered['total_area_km2'].round(3)

fig = make_subplots(rows=1, cols=2, column_widths=[0.8, 0.2], specs=[[{"type": "choropleth"}, {"type": "table"}]])


world_with_area_covered['hover_text'] = world_with_area_covered['name'] + '<br>Total Area: ' + world_with_area_covered['total_area_km2'].astype(str) + ' km²'

fig.add_trace(
    go.Choropleth(
        locations=world_with_area_covered['iso_a3'],
        z=world_with_area_covered['total_area_km2'],
        hoverinfo='text',
        text=world_with_area_covered['hover_text'], 
        colorscale=px.colors.sequential.Agsunset,
        colorbar_title="Total Area<br>km²",
    ),
    row=1, col=1
)


table_data = world_with_area_covered[['name', 'total_area_km2']].sort_values(by='total_area_km2', ascending=False)


def map_value_to_color(value, min_value, max_value, colorscale):
    normalized = (value - min_value) / (max_value - min_value)
    index = min(int(normalized * (len(colorscale) - 1)), len(colorscale) - 1)
    return colorscale[index]


min_area = table_data['total_area_km2'].min()
max_area = table_data['total_area_km2'].max()


colorscale = px.colors.sequential.Agsunset


row_colors = [map_value_to_color(value, min_area, max_area, colorscale) for value in table_data['total_area_km2']]

fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Total Area Covered (km²)"],
                    fill_color='#440154', 
                    font=dict(color='white'),
                    align='left'),
        cells=dict(values=[table_data.name, table_data.total_area_km2],
                   fill_color=row_colors,  
                   align='left')),
    row=1, col=2
)


fig.update_layout(
    title_text="Global Reforestation Area Covered (km²) with Data Table",
    showlegend=False,
    paper_bgcolor="lightgrey",
    title_x=0.5,
    width=1200,
    height=600,
    margin=dict(l=10, r=10, t=50, b=10)
)


fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.8, 0.2], 
                    specs=[[{"type": "choropleth"}, {"type": "table"}]])

fig.add_trace(
    px.choropleth(world_with_area_covered,
                  locations="iso_a3",
                  color="total_area_km2",
                  hover_name="name",
                  color_continuous_scale=px.colors.sequential.Agsunset).data[0],  
    row=1, col=1
)

table_data = world_with_area_covered[['name', 'total_area_km2']].sort_values(by='total_area_km2', ascending=False)

fig.add_trace(
    go.Table(
        header=dict(values=["Country", "Total Area Covered (km²)"],
                    fill_color='darkblue',  
                    font=dict(color='white'), 
                    align='left'),
        cells=dict(values=[table_data.name, table_data.total_area_km2],
                   fill_color='lightcyan',  
                   align='left')),
    row=1, col=2
)

fig.update_layout(
    title_text="Global Reforestation Area Covered (km²) with Data Table",
    showlegend=False,
    paper_bgcolor="lightgrey",  
)

fig.show()

In [ ]:
gdf_filtered=projects

In [ ]:



gdf_filtered['planting_date_reported'] = gdf_filtered['planting_date_reported'].astype(str)

gdf_filtered['year'] = gdf_filtered['planting_date_reported'].str.extract(r'(\d{4})')


gdf_filtered['year'] = pd.to_numeric(gdf_filtered['year'], errors='coerce')


year_counts = gdf_filtered['year'].value_counts()

print(year_counts)

In [ ]:
unique_years_count = gdf_filtered['year'].nunique()
print(unique_years_count)

In [ ]:



year_counts.sort_index(inplace=True)  

plt.figure(figsize=(10, 6)) 
plt.bar(year_counts.index.astype(str), year_counts.values)  
plt.xlabel('Year')  
plt.ylabel('Count') 
plt.title('Yearly Planting Date Reported Counts')  
plt.xticks(rotation=45)  
plt.show() 

In [ ]:


year_counts.sort_index(inplace=True)  


colors = plt.cm.viridis(np.linspace(0, 1, len(year_counts)))

plt.figure(figsize=(10, 6)) 
plt.bar(year_counts.index.astype(str), year_counts.values, color=colors)  
plt.xlabel('Year')  
plt.ylabel('Count')  
plt.title('Yearly Planting Date Reported Counts')  
plt.xticks(rotation=45) 
plt.show()  